# Neural Network-Based Image Processing on Flower Dataset

This notebook guides you through designing, implementing, and reporting on neural network-based techniques for image processing applications using a flower dataset.

## Outline
1. Import Required Libraries
2. Load and Explore the Flower Dataset
3. Preprocess Images
4. Build Neural Network Model
5. Train the Model
6. Evaluate Model Performance
7. Visualize Predictions
8. Generate Model Performance Report

In [4]:
# Install prerequisites
!pip install numpy pandas matplotlib tensorflow

  Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.5-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.59.1-cp312-cp312-win_amd64.whl.metadata (111 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using ca


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory


ModuleNotFoundError: No module named 'numpy'

## Load and Explore the Flower Dataset

Load your flower dataset, display sample images, and explore dataset characteristics such as class distribution and image dimensions.

In [ ]:
# Load and Explore the Flower Dataset
# Update 'data_dir' to the path where your flower dataset is stored

data_dir = 'path_to_flower_dataset'  # TODO: Update this path

# Load dataset using Keras utility
batch_size = 32
img_height = 180
img_width = 180

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
print('Classes:', class_names)

# Display sample images
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

## Preprocess Images

Resize images, normalize pixel values, and perform any necessary data augmentation or preprocessing steps.

In [ ]:
# Preprocess Images
# Normalize pixel values and apply data augmentation

normalization_layer = layers.Rescaling(1./255)

# Data augmentation
augmentation_layer = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def preprocess(ds):
    ds = ds.map(lambda x, y: (normalization_layer(x), y))
    ds = ds.map(lambda x, y: (augmentation_layer(x), y))
    return ds

train_ds = preprocess(train_ds)
val_ds = preprocess(val_ds)

## Build Neural Network Model

Define and implement a neural network architecture suitable for image classification or processing tasks using Keras.

In [ ]:
# Build Neural Network Model
num_classes = len(class_names)

model = keras.Sequential([
    layers.InputLayer(input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

## Train the Model

Train the neural network on the preprocessed flower dataset, including setting up training parameters and monitoring training progress.

In [ ]:
# Train the Model

epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

## Evaluate Model Performance

Assess the trained model using metrics such as accuracy, precision, recall, and confusion matrix on a validation or test set.

In [ ]:
# Evaluate Model Performance
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = []
y_pred = []
for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print('Classification Report:')
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(np.arange(len(class_names)), class_names, rotation=45)
plt.yticks(np.arange(len(class_names)), class_names)
plt.colorbar()
plt.show()

## Visualize Predictions

Display sample predictions from the model alongside the actual labels to visually inspect model performance.

In [ ]:
# Visualize Predictions
plt.figure(figsize=(12, 12))
for images, labels in val_ds.take(1):
    preds = model.predict(images)
    pred_labels = np.argmax(preds, axis=1)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        true_label = class_names[labels[i]]
        pred_label = class_names[pred_labels[i]]
        plt.title(f'True: {true_label}\nPred: {pred_label}')
        plt.axis("off")
plt.show()

## Generate Model Performance Report

Summarize the results, including key metrics, visualizations, and observations about the neural network's performance on the flower dataset.